#### Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Originally written in Mathematica by J. Christopher Edgar. Conversion to Jupyter Notebook by Song Liu.

The authors of this courseware are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.04 Convolution and Filtering - Spatial Domain</font>

# <font color=red>TUTORIALS</font>

### Setup

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
import matplotlib.patches as patches
import math
import cmath
import pandas as pd
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b",linewidth = 1): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))

## <font color=red>DSP.04.T1) Filtering Spatial Images</font>

### <font color=red>DSP.B1.T1.a) Loading and Displaying Grey Scale Images Files</font>

This first section repeats information already covered in Lesson 1. Skim through this first section and
then move on to new material.

Let's load a black-and-white image. Specifically, load in the 'statuesgreysmall.jpg' file.

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "statuesgreysmall.jpg"
file_path = filedialog.askopenfilename()
image = img.imread(file_path)
make_imshow(image)
plt.show()  

This picture was taken by David Edgar in Nassau, Bahamas. The statues are made from the trunks of trees.

This greyscale image is in what is called a JPEG format. As shown above, Python knows how to
display JPEG images. JPEG files typically have a filename extension of .jpeg or .jpg.

What is displayed as an image is really just a 2D matrix of numbers. Use the command below to obtain
the numerical values used to represent this image (be patient, it may take a few seconds to generate
the plot).

In [ ]:
image.shape 

In [ ]:
image_flattern = np.sort(image.flatten())
plt.plot(image_flattern)
plt.show()

The 2D matrix 'image' contains the values used to represent the image. The Python 'image.shape'
method indicates that the image is composed of a matrix containing 742×1111 = 824362 values. This
matrix is 742 rows by 1111 columns (notice that the width of the image (columns) is more than the
height (rows)). The graph above shows the range of values contained in the image. As indicated on the
y axis, the values range from 0 to 255. A '0' value represents pure black, a '255' value represents pure white,
and the values in-between represent shades of grey. The greyscale image shown above shows a full
range of greyscale values.

### <font color=red>DSP.04.T1.b) Filtering a Greyscale Image</font>

Load the image again.

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "statuesgreysmall.jpg"
file_path = filedialog.askopenfilename()
image = img.imread(file_path)
make_imshow(image)
plt.show()  

Filter this image. Remove high-frequency information. Start with a 2D kernel of size 25.

In [ ]:
kernel = np.array([[1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25]])

Convolve the kernel and the image to create a filtered image.

In [ ]:
from scipy import signal
                 
image_filtered = signal.convolve2d(image,kernel,boundary='symm', mode='same')
make_imshow(image_filtered)
plt.show() 

Compare the filtered and unfiltered image.

In [ ]:
plt.imshow(image, cmap='Greys_r')
make_imshow(image)
plt.show() 

make_imshow(image_filtered)
plt.show() 

What looks different? What information is missing in the filtered image?

The convolution applied a moving-average filter, which we know is a low-pass filter. So, it removed some high-spatial-frequency information. In other words, we blurred the image.

Try filtering the image with a larger kernel to remove more high-frequency information.

In [ ]:
from scipy import signal

kernel = 1/81 * np.ones((9,9))                 
image_filtered = signal.convolve2d(image,kernel,boundary='symm', mode='same')

Compare the filtered and unfiltered image.

In [ ]:
plt.imshow(image, cmap='Greys_r')
make_imshow(image)
plt.show() 

make_imshow(image_filtered)
plt.show() 

Only the broad outlines remain in the filtered image. Can you think of a reason to apply a low-pass filter to your own images? Although most of us like high definition TV, why do you think some TV personalities
are less excited about high definition TV? Do you think they might appreciate low-pass filters?

### <font color=red>DSP.04.T1.c) In the Above Example, What Frequencies Were Removed?</font>

Throughout Lesson 3, and in Lesson 4 Basics, we noted what frequency (or frequencies) were
removed when applying a kernel of a certain size. In the above example, however, when convolving the
data with different 3D kernels, we simply noted that larger 3D kernels removed more high-frequency
information (at the extreme, applying a kernel as large as the image, all information would be lost,
except the mean intensity value).

In the above example, why didn’t we specify which frequencies were removed?

Answer:
    
In the above example, determining what frequencies are removed is somewhat complex. To determine
the frequencies removed given a certain kernel, we’d first need to know the (spatial) sampling rate. Assuming
that we’re using a digital camera, the sampling rate is determined, in part, by the digital camera
megapixel count. The word ‘pixel’ comes from “picture element”. A 1MP camera has one million pixels (i.e., one million dots). The more pixels, in general, the better
the quality of the image (and the more expensive the camera).

In addition to knowing the camera pixel count, to determine the sample rate, we need to know the size
of the displayed or printed image. In particular, we need to know how many pixels (or dots) are used
per unit of distance in the horizontal dimension and ditto in the vertical dimension. This depends on the image size. Generally, people talk about dots per inch (dpi) or
pixels per inch (ppi) [or per cm, if you’re used to the metric system; 1 inche = 2.54 cm], and the more dots or pixels per inch
the higher the quality of the image. For good-quality images, a good guideline is 300 dpi, although this
depends a lot on what’s in the image. To achive 300 dpi for an 8" X 10" photo, you’d need a camera with at
least (8 × 300) + (10 × 300) = 7,200,000 or 7.2 megapixels. Thus, for any image, to determine the effect of
a given kernel, in addition to knowing the pixel count we need to know how densely the pixels are
packed for some unit of distance (here, how many dots per inch).

So, considering the above greyscale image, to determine what frequencies are removed when using a
particular 3D kernel, we need to know (1) the digital camera megapixel rate and (2) the size of the
image. In the above case, if we knew the dpi rate and the size of the image, using the procedures
covered in the Basics, we could determine the effect of a kernel of a given size. In this courseware, instead of
computing the above, we’ll simply qualitatively compare the image to determine whether high-frequency
or low-frequency information is removed.

The above is true for printed images. For images displayed on a video screen, the issues are more complex, since you also have to consider the
resolution of your computer screen.

## <font color=red>DSP.04.T2) Filtering RGB Images</font>

### <font color=red>DSP.04.T2.a) Loading and Displaying Color RGB Files</font>

Let's take a look at filtering color image. Before loading a color image, take a moment to understand an
important aspect of many digital color images.

Starting with three primary colors, these colors can be added together to produce an almost infinite
number of other colors. Start with the primary colors of light: red, green, and blue.
        
With colorants such as paint and ink, the three primary colors are
red, yellow, and blue, shaded with black and tinted with white.
Sometime folks also consider magenta and cyan to be primary colors for colorants.

In [ ]:
circle1 = plt.Circle((0.5, 0.5), 0.4, color='r')

fig, ax = plt.subplots() 
ax.add_patch(circle1)
plt.axis("off")

In [ ]:
circle2 = plt.Circle((0.5, 0.5), 0.4, color='g')
fig, ax = plt.subplots() 
ax.add_patch(circle2)
plt.axis("off")

In [ ]:
circle3 = plt.Circle((0.5, 0.5), 0.4, color='b')
fig, ax = plt.subplots() 
ax.add_patch(circle3)
plt.axis("off")

Python mixes these colors as follows:

In [ ]:
from numpy import random

r= random.rand()
g= random.rand()
b= random.rand()
rand_color = (r, g, b)

circle = plt.Circle((0.5, 0.5), 0.4, color=rand_color)
fig, ax = plt.subplots() 
ax.add_patch(circle)
plt.axis("off")

Rerun the previous code cell many times. You can experiment with mixing your own.

Whereas the colorants combine based on reflection and absorption of photons,
RGB depends on emission of photons from a compound excited to a higher energy state by impact with an electron beam (Wikipedia).

As we will see, many digital images are 'created' by recording how much red, green, and blue is needed
at each element of the digital image.

### <font color=red>DSP.04.T2.b) Creating a Color Image Using the RBG Color Model</font>

Load a color image. Specifically, load in the 'Augustcolor.jpg' file.

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "Augustcolor.jpg"
file_path = filedialog.askopenfilename()
image = img.imread(file_path)
make_imshow_color(image)
plt.show()  

Use the code below to examine the numerical values used to create the image.

In [ ]:
image.shape

In [ ]:
red_color = image[ : , : , 0]
green_color = image[ : , : , 1]
blue_color = image[ : , : , 2]

red_color = red_color.flatten()
green_color = green_color.flatten()
blue_color = blue_color.flatten()
  
# Separate Histograms for each color
plt.subplot(3, 1, 1) 
plt.title("histogram of Red") 
plt.hist(red_color, color="red")  
  
plt.subplot(3, 1, 2) 
plt.title("histogram of Green") 
plt.hist(green_color, color="green") 
  
plt.subplot(3, 1, 3) 
plt.title("histogram of Blue") 
plt.hist(blue_color, color="blue")
  
# for clear view 
plt.tight_layout() 
plt.show() 
  
# combined histogram 
plt.title("Histogram of all RGB Colors") 
plt.hist(red_color, color="red") 
plt.hist(green_color, color="green")
plt.hist(blue_color, color="blue")  

plt.show() 

Notice that the Python ‘shape’ command indicates that this is a 3D matrix (the greyscale
image was a single layer 2D matrix). The color 3D maxtix has three layers (channels), and it's represented
in what's known as an RGB format. The letters in RGB refer to 'Red', 'Green', and 'Blue'. The rows and
columns contain information on how much of each specific color is contained at each pixel. The first
two dimensions tell us that the image is composed of 480 × 320 = 153600 pixels per layer of the image
(i.e., for each color).

Each layer in the matrix (the third dimension) represents a separate color - one layer each for R, G, and
B. As shown above, the RGB color model is an additive model in which red, green, and blue are combined
in various ways to reproduce other colors. At each displayed pixel, the information contained in
the 3D matrix provides the information on how much of each primary color is applied at each pixel. Depending on your field and software, the RGB values are scaled from 0 to 1 or from 0 to 255. In this courseware, the RGB values are often scaled from 0 to 1, but sometimes 0 to 255. If the range is 0 to 255, a value of 0 indicates no blue (most of the tree), and a value of 255 indicates the most blue (pants of course!). 

By the way, if you use 8 bits to store red, 8 to store green, and 8 to store blue, then you'll use 24 bits for each pixel, and you'll be able to store any of over 16 million colors.

### <font color=red>DSP.04.T2.c) Taking about RBG images</font>

Take a look at the above information again.

In [ ]:
image.shape

In [ ]:
red_color = image[ : , : , 0]
green_color = image[ : , : , 1]
blue_color = image[ : , : , 2]

red_color = red_color.flatten()
green_color = green_color.flatten()
blue_color = blue_color.flatten()
  
# Separate Histograms for each color
plt.subplot(3, 1, 1) 
plt.title("histogram of Red") 
plt.hist(red_color, color="red")  
  
plt.subplot(3, 1, 2) 
plt.title("histogram of Green") 
plt.hist(green_color, color="green") 
  
plt.subplot(3, 1, 3) 
plt.title("histogram of Blue") 
plt.hist(blue_color, color="blue")
  
# for clear view 
plt.tight_layout() 
plt.show() 
  
# combined histogram 
plt.title("Histogram of all RGB Colors") 
plt.hist(red_color, color="red") 
plt.hist(green_color, color="green")
plt.hist(blue_color, color="blue")  

plt.show() 

You can also see that image is represented here as a 480 X 320 X 3 array, with each pixel represented as
an array of {r,g,b,} triples. Because the red, green, and blue information is individually represented,
each component can be separately viewed.

Here are just the red values.

In [ ]:
redimage = image[ : , : , 0]
redimage

Each of the displayed pixels shows how much red is added to the composite image. A value of 0 indicates
no red (e.g., the shadows). A value of 255 indicates the most red.

Show the green values.

In [ ]:
greenimage = image[ : , : , 1]
greenimage

Each of the displayed pixels shows how much green is added to the composite image. A value of 0
indicates no green (shoes). A value of 255 indicates the most green (leaves in tree).

Show the blue values.

In [ ]:
blueimage = image[ : , : , 2]
blueimage

Each of the displayed pixels shows how much blue is added to the composite image. A value of 0
indicates no blue (most of the tree). A value of 255 (or 1) indicates the most blue (pants of course!).
The three primary colors are combined to produce a single image.

Spend a moment examining how the primary colors are combined to create the displayed image. Notice, for example, that the sky is made
from both green and blue. Notice that the yellow shirt is made by combining red and green.

### <font color=red>DSP.04.T2.d) Filtering a Color Image</font>

Earlier in this Tutorial, with a greyscale image, the kernel was convolved with a single 2D matrix of values representing the image.
A RGB image represents the image using three separate matrices. As such, some decisions need to be
made. Should the kernel be convolved with all three matrices (equivalently, a 3D matrix)? With just the red matrix? Just the blue
matrix?

Convolve a kernel only with the matrix containing the red information.

In [ ]:
kernel = np.array([[1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25],
                  [1/25, 1/25, 1/25, 1/25, 1/25]])

The entire image can be thought of as being represented as a 3D matrix of values. The rows and
columns contain information on how much of each color is applied at each pixel in the image. Three 2D
layers provide information about each color (one layer each for R, G, and B). Apply the kernel to the
layer containing information about red coloring by grabbing the first layer.

In [ ]:
from scipy import signal
                 
redimage_filtered = np.round(signal.convolve2d(redimage,kernel,boundary='symm', mode='same'))

make_imshow(redimage)
plt.show()

make_imshow(redimage_filtered)
plt.show()

Now do the same for the green layer.

In [ ]:
from scipy import signal
                 
greenimage_filtered = np.round(signal.convolve2d(greenimage,kernel,boundary='symm', mode='same'))

make_imshow(greenimage)
plt.show()

make_imshow(greenimage_filtered)
plt.show()

Now do the same for the blue layer.

In [ ]:
from scipy import signal
                 
blueimage_filtered = np.round(signal.convolve2d(blueimage,kernel,boundary='symm', mode='same'))

make_imshow(blueimage)
plt.show()

make_imshow(blueimage_filtered)
plt.show()

Now look at the composite image with the filtered red, green, and blue components.

In [ ]:
image_filtered = np.zeros((480, 320, 3), 'uint8')
image_filtered[...,0] = redimage_filtered
image_filtered[...,1] = greenimage_filtered
image_filtered[...,2] = blueimage_filtered

make_imshow_color(image)
plt.show()

make_imshow_color(image_filtered)
plt.show()

In the 'Give it a Try' sections of this lesson, you'll see what the image looks like when you filter just one of the primary
colors.